# Load data in Google Big Query

## Using Terminal

### Create new dataset in existing project

In [ ]:
!export PROJECT=artful-talon-355716
!export DATASET=rex_ai
!export LOCATION=EU

Command below should output `Dataset 'artful-talon-355716:rex_ai' successfully created.`

In [ ]:
!bq mk --sync --project_id $PROJECT --data_location $LOCATION $DATASET

### Create new table in dataset

In [1]:
!export REGION=europe-southwest1
# europe-southwest1 (Madrid)
!export TABLE=idxs

In [ ]:
!bq mk --sync --location=$REGION $DATASET.$TABLE

## Using Python

In [1]:
import pandas as pd
from google.cloud import bigquery

In [2]:
project_id = 'artful-talon-355716'
dataset_id = 'rex_ai'
location = 'EU'

client = bigquery.Client()
dataset_id = f'{project_id}.{dataset_id}'
dataset = bigquery.Dataset(dataset_id)
dataset.location = location

In [7]:
dataset = client.create_dataset(dataset, timeout=30)
print(f'Created dataset {project_id}.{dataset_id}')

Created dataset artful-talon-355716.artful-talon-355716.rex_ai_test


In [8]:
df = pd.read_csv('../../model/db/data/merge/secondary/idxs_.csv', index_col=0)
df.index = pd.to_datetime(df.index)
df.index.name = 'Datetime'
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 105840 entries, 2005-01-03 00:00:00+00:00 to 2022-09-02 23:00:00+00:00
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   AUD     105840 non-null  float64
 1   CAD     105840 non-null  float64
 2   CHF     105840 non-null  float64
 3   EUR     105840 non-null  float64
 4   GBP     105840 non-null  float64
 5   JPY     105840 non-null  float64
 6   NZD     105840 non-null  float64
 7   USD     105840 non-null  float64
dtypes: float64(8)
memory usage: 7.3 MB


In [3]:
df.head()

NameError: name 'df' is not defined

In [10]:
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
job = client.load_table_from_dataframe(
    df, f'{dataset_id}.idxs_', job_config=job_config
)  # Make an API request.
job.result()

LoadJob<project=artful-talon-355716, location=EU, id=a5e530b3-6c35-4007-baf3-abc7dbe2efce>

## Upload all data

In [3]:
import os
import time

In [14]:
def upload_data(path, dataset_id):
    
    try:
        client.create_dataset(bigquery.Dataset(dataset_id), timeout=30)
        print(f'Created dataset {dataset_id}', flush=True, end='\r')
    except Exception:
        print(f'Dataset {dataset_id} already exists', flush=True, end='\r')
        
    for file in os.listdir(path):    
        
        start_time = time.time()
        print(f'Uploading {file} to {dataset_id}', flush=True, end='\r')
        data = pd.read_csv(os.path.join(path, file), index_col=0)
        data.index = pd.to_datetime(data.index)
        data.index.name = 'Datetime'
        
        
        try:
            job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
            job = client.load_table_from_dataframe(
                data, f"{dataset_id}.{file.split('.')[0]}", job_config=job_config
            )  # Make an API request.
            job.result()
        except Exception as e:
            print(e)
        
        print(f'Uploaded {file} in {round(time.time() - start_time, 2)}s')
        
    return

In [15]:
PRIMARY_PATH = os.path.join(os.path.abspath('../../'), 'model', 'db', 'data', 'merge', 'primary')
upload_data(PRIMARY_PATH, dataset_id)

Upload results:csv to artful-talon-355716.rex_ai_ai
Uploaded spreads.csv in 78.69s
Uploaded volumes.csv in 17.92s
Uploaded asks.csv in 32.45s
Uploaded bids.csv in 24.39s
Uploaded mids.csv in 32.76s


In [17]:
SECONDARY_PATH = os.path.join(os.path.abspath('../../'), 'model', 'db', 'data', 'merge', 'secondary')
upload_data(SECONDARY_PATH, dataset_id)

In [5]:
data = client.query(f'SELECT * FROM {dataset_id}.rets_ ORDER BY Datetime ASC LIMIT 1000').to_dataframe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   Datetime  1000 non-null   datetime64[ns, UTC]
 1   AUD_CAD   991 non-null    float64            
 2   AUD_CHF   991 non-null    float64            
 3   AUD_JPY   991 non-null    float64            
 4   AUD_NZD   991 non-null    float64            
 5   AUD_USD   991 non-null    float64            
 6   CAD_CHF   991 non-null    float64            
 7   CAD_JPY   991 non-null    float64            
 8   CHF_JPY   991 non-null    float64            
 9   EUR_AUD   991 non-null    float64            
 10  EUR_CAD   991 non-null    float64            
 11  EUR_CHF   991 non-null    float64            
 12  EUR_GBP   991 non-null    float64            
 13  EUR_JPY   991 non-null    float64            
 14  EUR_NZD   991 non-null    float64            
 15  EUR_USD   991 non-null

In [ ]:
data.info()